# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn import ensemble, model_selection, metrics, datasets
import numpy as np
import pandas as pd

In [2]:
X, y = datasets.load_boston(return_X_y=True)

In [3]:
p = 0.75

idx = int(p * X.shape[0]) + 1

X_train, X_test = np.split(X, [idx])
y_train, y_test = np.split(y, [idx])

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [4]:
def L_derivative(y_train, z):
    return (y_train - z)

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [5]:
from sklearn.tree import DecisionTreeRegressor

In [6]:
base_algorithms_list = []
coefficients_list = []
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [7]:
z = np.zeros( (y_train.shape) )
for i in range(50) :
    coefficients_list.append(0.9)
    regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
    regressor.fit(X_train, L_derivative(y_train, z))
    base_algorithms_list.append(regressor)
    z = gbm_predict(X_train)

In [8]:
RMSE = metrics.mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5

In [9]:
with open('grad_boosting_answer2.txt', 'w') as fout:
    fout.write(str(RMSE))

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [10]:
base_algorithms_list3 = []
coefficients_list3 = []
def gbm_predict3(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list3, coefficients_list3)]) for x in X]

In [11]:
z = np.zeros( (y_train.shape) )
for i in range(50) :
    coefficients_list3.append(0.9 / (1.0 + i))
    regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
    regressor.fit(X_train, L_derivative(y_train, z))
    base_algorithms_list3.append(regressor)
    z = gbm_predict(X_train)

In [12]:
RMSE3 = metrics.mean_squared_error(y_test, gbm_predict3(X_test)) ** 0.5

In [13]:
with open('grad_boosting_answer3.txt', 'w') as fout:
    fout.write(str(RMSE3))

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [14]:
import xgboost as xgb

In [15]:
grad_boosting = xgb.XGBRegressor(objective ='reg:squarederror')

In [16]:
param_range = np.array([10, 20, 30, 40, 50, 100, 150, 200, 300])
train_scores, test_scores = model_selection.validation_curve(grad_boosting, X, y, param_name="n_estimators", param_range=param_range, cv=3, scoring="neg_mean_squared_error")

In [17]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
print(train_scores_mean, test_scores_mean)

[-82.6570422  -16.33599234  -6.21645458  -3.99816371  -3.18283065
  -1.68790195  -1.03552382  -0.68282062  -0.33472582] [-105.43344766  -42.26776885  -31.0001559   -28.07056837  -27.03862157
  -26.30337807  -26.18035887  -26.06885546  -26.49503566]


In [18]:
param_range = np.array([3, 5, 7, 10, 20, 50, 100])
train_scores, test_scores = model_selection.validation_curve(grad_boosting, X, y, param_name="max_depth", param_range=param_range, cv=3, scoring="neg_mean_squared_error")

In [19]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
print(train_scores_mean, test_scores_mean)

[-1.68790195e+00 -2.84110830e-01 -3.34000775e-02 -2.41735183e-03
 -4.12285490e-04 -4.03187177e-04 -4.03187177e-04] [-26.30337807 -25.67920757 -26.3740079  -26.82352813 -26.52167108
 -26.56821707 -26.56821707]


In [20]:
answer = str(2) + ' ' + str(3)
with open('grad_boosting_answer4.txt', 'w') as fout:
    fout.write(answer)

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [24]:
RMSE5 = metrics.mean_squared_error(y_test, regressor.predict(X_test)) ** 0.5

In [25]:
with open('grad_boosting_answer5.txt', 'w') as fout:
    fout.write(str(RMSE5))